In [1]:
import sys, os, time
import numpy as np
%load_ext autoreload
%autoreload 2
%matplotlib inline
#%matplotlib tk
import matplotlib.pyplot as plt

import tnt_util as util
from tnt_util import adict, idict, xset, collate, load, render_dict, get_object, save, Logger, DigitalLog
from tnt_setup import init_gamestate, setup_phase
from tnt_cards import load_card_decks, draw_cards
from collections import namedtuple

from flask_app import *

In [2]:
ping()

'Backend active: use "init" to init game'

In [3]:
out = init_game()
out

Beginning phase: Setup
Beginning phase: New_Year


TypeError: 'NoneType' object is not callable